<a href="https://colab.research.google.com/github/RuslanFF1968/RuslanFF1968/blob/main/Clustering_Case.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import of libraries

In [ ]:
from google.colab import files
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import utils
import os

from tensorflow.keras.models import Model

from tensorflow.keras.layers import Dense, Dropout,SpatialDropout1D, BatchNormalization, Embedding, Flatten, Input

from keras.preprocessing.text import Tokenizer

%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


#Data uploading

In [ ]:
!rm -R /content/texts

rm: cannot remove '/content/texts': No such file or directory


In [ ]:
# Writing the path to the base in the Google Drive, creating the folder texts and extracting from there
!unzip -q '/content/drive/My Drive/Базы/Тексты писателей.zip' -d /content/texts

unzip:  cannot find or open /content/drive/My Drive/Базы/Тексты писателей.zip, /content/drive/My Drive/Базы/Тексты писателей.zip.zip or /content/drive/My Drive/Базы/Тексты писателей.zip.ZIP.


In [ ]:
def readText(fileName): # Declearing the functions for file reading. To the input the path to file is sent
  f = open(fileName, 'r')        # Setting the opening of the desired file in the read mode
  text = f.read()                # Reading the text
  text = text.replace("\n", " ") # Line breaks converting in spaces

  return text                    # Returing the file text

# Declaring the classes which are of interest
className = ["О. Генри", "Стругацкие", "Булгаков", "Саймак", "Фрай", "Брэдберри"]
nClasses = len(className) # Counting the number of classes

In [ ]:
!ls

drive  sample_data


In [ ]:
# Загружаем обучающие тексты

trainText = [] # Forming the training texts
testText = []  # Forming the test texts

# The formation shall be implemented as follows
# The class of each i-th element in the training sampling must correspond to
# to the class of each i-th element in the test sampling

for i in className:              # Go through each class
  for j in os.listdir('texts/'): # Go through each file in the folder with texts #
    if i in j:                   # Checking if the file j contains the name of the class i in the name

      if 'Обучающая' in j:                       # Если в имени найденного класса есть строка "Обучающая"
        trainText.append(readText('texts/' + j)) # добавляем в обучающую выборку
        print(j, 'добавлен в обучающую выборку') # Выводим информацию
      if 'Тестовая' in j:                        # Если в имени найденного класса есть строка "Тестовая"
        testText.append(readText('texts/' + j))  # добавляем в обучающую выборку
        print(j, 'добавлен в тестовую выборку')  # Выводим информацию
  print()

FileNotFoundError: [Errno 2] No such file or directory: 'texts/'

In [ ]:
print(len(trainText))    # Количество элементов в trainText
print(len(trainText[0])) # Количество символов в первом источнике в trainText

#Обработка данных


In [ ]:
maxWordsCount = 30000 # Определяем максимальное количество слов/индексов, учитываемое при обучении текстов

# Воспользуемся встроенной в Keras функцией Tokenizer для разбиения текста и превращения в матрицу числовых значений
# num_words=maxWordsCount - определяем максимальное количество слов/индексов, учитываемое при обучении текстов
# filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n' - избавляемся от ненужных символов
# lower=True - приводим слова к нижнему регистру
# split=' ' - разделяем слова по пробелу
# char_level=False - просим токенайзер не удалять однобуквенные слова
tokenizer = Tokenizer(num_words=maxWordsCount, filters='!"#$%&()*+,-–—./…:;<=>?@[\\]^_`{|}~«»\t\n\xa0\ufeff', lower=True, split=' ', oov_token='unknown', char_level=False)

tokenizer.fit_on_texts(trainText) # "Скармливаем" наши тексты, т. е. даём в обработку методу, который соберет словарь частотности
items = list(tokenizer.word_index.items()) # Вытаскиваем индексы слов для просмотра

In [ ]:
print(items[:50])                   # Посмотрим 50 самых часто встречающихся слов
print("Размер словаря", len(items)) # Длина словаря

In [ ]:
# Преобразовываем текст в последовательность индексов согласно частотному словарю
trainWordIndexes = tokenizer.texts_to_sequences(trainText) # Обучающие тесты в индексы
testWordIndexes = tokenizer.texts_to_sequences(testText)   # Проверочные тесты в индексы

print("Взглянем на фрагмент обучающего текста:")
print("В виде оригинального текста:              ", trainText[1][:87])
print("Он же в виде последовательности индексов: ", trainWordIndexes[1][:20], '\n')

In [ ]:
print("Статистика по обучающим текстам:")

symbolsTrainText = 0 # Объявляем переменную для подсчета символов в обучающих текстах
wordsTrainText = 0   # Объявляем переменную для подсчета слов в обучающих текстах

for i in range(nClasses): # Проходим по всем классам
  print(className[i], " "*(10-len(className[i])), len(trainText[i]), "символов, ", len(trainWordIndexes[i]), "слов")
  symbolsTrainText += len(trainText[i])      # Считаем символы
  wordsTrainText += len(trainWordIndexes[i]) # Считаем слова

print('----')
print("В сумме ", symbolsTrainText, " символов, ", wordsTrainText, " слов \n")
print()
print("Статистика по тестовым текстам:")

symbolsTestText = 0 # Объявляем переменную для подсчета символов в тестовых текстах
wordsTestText = 0   # Объявляем переменную для подсчета слов в тестовых текстах

for i in range(nClasses): # Проходим по всем классам
  print(className[i], ' '*(10-len(className[i])), len(testText[i]), "символов, ", len(testWordIndexes[i]), "слов")
  symbolsTestText += len(testText[i])      # Считаем символы
  wordsTestText += len(testWordIndexes[i]) # Считаем слова
print('----')
print("В сумме ", symbolsTestText, " символов, ", wordsTestText, " слов")

#Создание обучающей и проверочной выборки

**Функции для формирования выборки по отрезкам текста с заданным шагом**

In [ ]:
###########################
# Формирование обучающей выборки по листу индексов слов
# (разделение на короткие векторы)
##########################
def getSetFromIndexes(wordIndexes, xLen, step): # wordIndexes - список индексов, xLen - длина отрезка разбиения, step - шаг разбиения
  xSample = []                # Создаем пустой список под результат
  wordsLen = len(wordIndexes) # Получаем длину списка индексов
  index = 0 # Задаем переменную, в которой будет храниться стартовый индекс для очередного "куска" разбиения

  #Идём по всей длине вектора индексов
  #"Откусываем" векторы длины xLen и смещаемся вперёд на step

  while (index + xLen <= wordsLen):               # Если можно получить очередной "кусок"
    xSample.append(wordIndexes[index:index+xLen]) # Добавляем в результирующий список элементы из wordIndexes, начиная с элемента с индексом index, заканчивая индексом index+xLen
    index += step                                 # Увеличиваем index на величину шага

# Таким образом на 0 итерации в результат добавятся элементы с 0 по 3000 (index=0, index + xLen = 3000)
# На первой итерации index увеличиваем на 100 и врезультат добавятся элементы с 100 по 1100 (index = 100, index + xLen = 1100)
# И так до тех пор, пока index + xLen не станет больше длины входного списка индексов wordsLen

  return xSample # Возвращаем сформированный список

In [ ]:
###########################
# Формирование обучающей и проверочной выборки
# Из двух листов индексов от двух классов
##########################
def createSetsMultiClasses(wordIndexes, xLen, step): # wordIndexes - список индексов для всех классов, xLen - длина отрезка разбиения, step - шаг разбиения

  # Для каждого из 6 классов
  # Создаём обучающую/проверочную выборку из индексов
  nClasses = len(wordIndexes) # Задаем количество классов выборки
  classesXSamples = []        # Создаем пустой список под выборки для каждого из классов
  for wI in wordIndexes:      # для каждого текста выборки из последовательности индексов
    classesXSamples.append(getSetFromIndexes(wI, xLen, step)) # добавляем в список очередную выборку
  # В итоге classesXSamples будет иметь nClasses-элементов, где каждый элемент представляет собой список, который возвращает функция getSetFromIndexes

  # Формируем один общий xSamples
  x_train = [] # Создаем пустой список под x_train
  y_train = [] # Создаем пустой список под y_train

  for t in range(nClasses):   # Проходим по всем имеющимся классам
    currY = utils.to_categorical(t, nClasses) # Tекущий класс переводится в вектор длиной 6 вида [0.0.0.1.0.0.]
    xT = classesXSamples[t]   # Берем выборку для очередного класса
    for i in range(len(xT)):  # Проходим по всем элементам выборки
      x_train.append(xT[i])   # Добавляем в x_train
      y_train.append(currY)   # Добавляем соответствующий вектор класса

  x_train = np.array(x_train) # Переводим в массив numpy для подачи в нейронку
  y_train = np.array(y_train) # Переводим в массив numpy для подачи в нейронку

  return (x_train, y_train)   # Функция возвращает выборку и соответствующие векторы классов

**Подготовка данных**

In [ ]:
# Задаём базовые параметры
xLen = 3000 # Длина отрезка текста, по которой анализируем, в словах
step = 100  # Шаг разбиения исходного текста на обучающие вектора

In [ ]:
# Формируем обучающую и тестовую выборку
xTrain, yTrain = createSetsMultiClasses(trainWordIndexes, xLen, step) # Извлекаем обучающую выборку
xTest, yTest = createSetsMultiClasses(testWordIndexes, xLen, step)    # Извлекаем тестовую выборку

# Отображаем размерности
print(xTrain.shape)
print(yTrain.shape)
print(xTest.shape)
print(yTest.shape)

**Recognition of the test sampling**


In [ ]:
###########################
# Представляем тестовую выборку в удобных для распознавания размерах
##########################
def createTestMultiClasses(wordIndexes, xLen, step): #функция принимает последовательность индексов, размер окна, шаг окна
  # Для каждого из 6 классов
  # Создаём тестовую выборку из индексов
  nClasses = len(wordIndexes) # Получаем количество классов
  xTest6Classes = []          # Создаем пустой список под результат
  for wI in wordIndexes:      # Проходим по всем текстам из входного набора wordIndexes
    sample = (getSetFromIndexes(wI, xLen, step)) # Разбиваем очерденой текст на отрезки длиной xLen с шагом step
    xTest6Classes.append(np.array(sample))       # Добавляем полученный список к результату
  return xTest6Classes        # Возвращаем сформированный список

###########################
# Распознаём тестовую выборку и выводим результаты
##########################
def recognizeMultiClass(model, xTest, modelName):
  print("НЕЙРОНКА: ", modelName) # Отображаем имя нейронки
  print()

  val = []  # Создаем пустой список под результат
  pred = [] # Создаем пустой список для хранений распознанных значений отдельных "кусков"

  totalSumRec = [] # Создаем пустой список под результат

  for i in range(nClasses):            # Проходим по всем классам
    currPred = model.predict(xTest[i]) # Распознаем i-тый элемент тестовой выборки (xTest[i] - список, который создается функцией createTestMultiClasses для i-го тестового набора)
    pred.append(currPred)              # Добавляем распознанный список значений в pred
    currOut = np.argmax(currPred, axis=1) # Опрделяем номер распознаного класса для каждого элемента из currPred

    # Считаем процент распознавания каждого класса за каждый
    # Получаем матрицу 6 на 6
    evVal = []
    sumRec = []
    for j in range(nClasses): # Проходим по всем классам
      sumRec.append(0)
      for t in range(len(currOut)): # Проходим по всему количество элементу в currOut (равно количество кусков, на которые был разделен исходных тестовый набор)
        if (currOut[t] == j):       # Если currOut[t] (класс, к которому модель отнесла данный кусок) равен текущему классу j
          sumRec[j] += 1            # То увеличиваем на 1 сумму sumRec для текущего класса
      evVal.append(sumRec[j] / len(currOut)) # Считаем evVel для текущего класса (количечство элеметнов, которые были отнесены к классу j делим на общее количество элементов)

    # В итоге evVal будет длиной равной количеству классу, где i-тый элемент показывает вероятность принадлежности к i-му классу
    # Например evVal = [0.79, 0.02, 0.13, 0.05, 0.03, 0.1] означает, что 79% всех элементов в xTest[i] были отнесены к первому классу писателей и т. д.
    totalSumRec.append(sumRec[i])   # Добавляем в totalSumRec количество вернораспознанных элементов sumRec[i] для i-го класса

    # Определяем, какой в класс в итоге за какой был распознан
    val.append(evVal[i]) # Добавляем процент для i-го класса из evVal в массив val
    recognizedClass = np.argmax(evVal) # Находим индекс максимального элемента в evVal (соответсвует номеру класса)

    # Выводим результаты распознавания по текущему классу
    isRecognized = "распознано НЕ ВЕРНО!" # По умолчанию "неверно"
    if (recognizedClass == i):            # Если индекс максимального элемента в evVal равен текущему классу i, то
      isRecognized = "распознано ВЕРНО!"  # распознано верно
    print(className[i], " распознано ", int(100*evVal[i]), "% сеть считает, что это ", className[recognizedClass], ", ", isRecognized) # Выводим информацию о распознавании

  # Выводим средний процент распознавания по всем классам вместе
  print()
  sumVal = 0
  sumCount = 0
  lenXtest = []
  for i in range(nClasses):
    sumCount += len(xTest[i]) # Увеличиваем общее количество элементов на длину len(xTest[i])
    sumVal += totalSumRec[i]  # Увеличиваем количество верных ответов на totalSumRec[i]
  print("Средний процент распознавания ", int(100*sumVal/sumCount), "%") # Отображаем средний процент для всех примеров в тестовой выборке

  print()

  return sumVal/sumCount # Возвращаем средний процент распознавания

#Embedding


**Embedding + Dense**

In [ ]:
# Создаём сеть с Embedding слоем
text_input = Input((xTrain.shape[1],)) # Задаем входной слой для нейронки (размерность входных данных равна xTrain.shape[1] (в нашем примере 3000))
emb = Embedding(maxWordsCount, 100, input_length=xLen)(text_input) # Добавляем Embedding-слой с размерностью векторного прстранства 100 (выбрано тестовым путем) и сохраняем в переменнуд emb

In [ ]:
# Создаем стандартную полносвязную сеть для классификатора
x = SpatialDropout1D(0.25)(emb)
x = Flatten()(x)
x = BatchNormalization()(x)
x = Dense(200, activation="relu")(x)
x = Dropout(0.25)(x)
x = BatchNormalization()(x)
output = Dense(6, activation='softmax')(x) # Добавляем Dense-слой с softmax-Активацией и 6 нейронами

modelE = Model(text_input, output) # Создаем модель классификатора со входм text_input и выходом output

# Компилируем модель
modelE.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

#Обучаем сеть на xTrain-наборе
history = modelE.fit(xTrain,
                    yTrain,
                    epochs=30,
                    batch_size=200,
                    validation_data=(xTest, yTest))

# Отображаем графики для acc и vall_acc
plt.plot(history.history['accuracy'],
         label='Доля верных ответов на обучающем наборе')
plt.plot(history.history['val_accuracy'],
         label='Доля верных ответов на проверочном наборе')
plt.xlabel('Эпоха обучения')
plt.ylabel('Доля верных ответов')
plt.legend()
plt.show()

In [ ]:
modelE.summary()

In [ ]:
# Проверяем результаты работы Ebedding сети
xTest6Classes = createTestMultiClasses(testWordIndexes, xLen, step)
# Отобразим размерности тестовых текстов, разбитых на куски фиксированной длины
for i in range(nClasses):
  print(str(len(xTest6Classes[i])) + ' x ' + str(len(xTest6Classes[i][0])))

In [ ]:
# Отобразим статистику распознавания по писателям
pred = recognizeMultiClass(modelE, xTest6Classes, "Embedding + Dense")

#Кластеризация

In [ ]:
# Создадим модель, которая на входе дает данные с embedding'а
model2 = Model(text_input, emb)


In [ ]:
model2.summary()

In [ ]:
# Назначим нашей модели веса из обученной ранее модели классификатора
model2.layers[0] = modelE.layers[0] # Веса входного слоя нашей модели соответствуют весам входного слоя модели классификатора
model2.layers[1] = modelE.layers[1] # Веса Embedding слоя нашей модели соответствуют весам Embedding слоя модели классификатора

In [ ]:
allwords = np.array([it[0] for it in items])   # Создадим список слов из нашего словаря items
allindexes = np.array([it[1] for it in items]) # Создадим список индексов из нашего словаря items

In [ ]:
print(allwords[:20])   # Отобразим первый 20 слов
print(allindexes[:20]) # Отобразим первые 20 индексов

In [ ]:
len(allwords)

In [ ]:
# Так как наша модель требует входные данные размерностью 3000
# Требуется преобразовать массив индексов allindexes таким образом, чтобы длина каждого из его элементов была равна 3000

# Один из способов реализовать это: размножить каждый элемент 3000 раз
words = np.array([np.array([allindexes[t] for i in range(3000)]) for t in range(5000)])

In [ ]:
print(words.shape) # Отобразим размерность words

In [ ]:
print(words[0, :20]) # Отобразим первые 20 элементов 0го элемента
print(words[1, :20]) # Отобразим первые 20 элементов 1го элемента

In [ ]:
outall = model2.predict(words) # Теперь можно вызывать predict для массива words

In [ ]:
print(outall.shape) # Отобразим размерность outall

In [ ]:
# Так как в words все значения одной строки одинаковые (1,1,1,1... или 2,2,2,2.... и т.д.)
# То и outall все значения одной строки будут одинаковыми
# Поэтому без проблем можем оставить только 0-ые элементы из outall
outall2 = np.array([o[0] for o in outall])

In [ ]:
print(outall2.shape) # Выведем размерность outall2

In [ ]:
print(outall2[0]) # Отобразим данные 0го элемента
# Они представляют из себя 100мерный вектор
# Этот вектор представляет собой закодированное слово "и" (1-е слово в нашем словаре)
# Во втором элементе будет храниться закодированное 100мерным вектором 2-е слово в словаре: "в"

In [ ]:
from sklearn.cluster import KMeans # Подключаем модуль Kmeans
import time # Подключаем модуль time

In [ ]:
current_time = time.time()    # Засекаем текщуее время начала операции
clustersCount = 100           # Задаем количество кластеров (выбрано произвольным путем из расчета того, что всего 20 000 слов берем)
kmean = KMeans(clustersCount) # Создаем экземпляр KMeans и указываем количество классов
kmean.fit(outall2)            # Делаем обработку по Embedding-данным из outall2
labels = kmean.labels_        # Запоминаем метки классов в labels
print ('Время обработки:' + str(round(time.time() - current_time)) + 'с.') # Выводим время работы алгоритма

In [ ]:
print(labels[:20]) # Отобразим первые 20 меток

In [ ]:
start_allwords = allwords[:5000] # Возьмем первые 5 000 слов из списка всех слов allwords

In [ ]:
print(start_allwords[labels==1]) # Отобразим, какие слова из start_allwords попали в 1 кластер

In [ ]:
# Выведем 100 кластеров
for i in range(100):
  print(i, start_allwords[labels==i])